# Tercera parte.  Recomendacion basada en filtrado colaborativo.

**Grupo 16**
- Daniela Alejandra Cordova Porta
- David Bugoi
- Erik Karlgren Domercq

En esta tercera parte utilizaremos la librería SURPRISE Se puede consultar la documentacion en http://surpriselib.com/

Para instalarla: `conda install -c conda-forge scikit-surprise` o `pip install numpy pip install scikit-surprise`.

La librería SurPRISE (<i>Simple Python RecommendatIon System Engine</i>) tiene algoritmos de predición de ratings incluidos: <i>baseline algorithms</i>, <i>neighborhood methods</i>, <i>matrix factorization-based</i> (SVD, PMF, SVD++, NMF) y otros. También tiene predefinidas las medidas de similitud más comunes sobre vectores (<i>cosine</i>, MSD, pearson…) Una de las cosas más útiles es que proporciona herramientas para evaluar, analizar y comparar el rendimiento de distintos algoritmos. Lo que vamos a hacer en esta parte de la práctica es probar varios procedimientos de evaluación cruzada midiendo datos sobre errores entre el valor real (conocido) y la predicción del recomendador. Las siglas corresponden a las siguientes medidas:

- MAE: _Mean Absolute Error_
- RMSE: _Root mean square error_ (RMSE)
- MSE: mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. -square root of the mean square error.

Vamos a ejecutar algunos recomendadores y evaluarlos para poder comentar los resultados.

In [1]:
from collections import defaultdict
import numpy as np

from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
## Ejemplo getting started de la documentación de SURPRISE
##http://surpriselib.com/

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9308  0.9329  0.9350  0.9508  0.9308  0.9361  0.0075  
MAE (testset)     0.7338  0.7360  0.7389  0.7491  0.7370  0.7390  0.0053  
Fit time          3.96    4.01    4.17    4.14    5.21    4.30    0.46    
Test time         0.17    0.17    0.13    0.17    0.37    0.20    0.09    


{'test_rmse': array([0.93080079, 0.93289501, 0.93504822, 0.95077306, 0.93083897]),
 'test_mae': array([0.73378163, 0.73595041, 0.73893383, 0.74909009, 0.73703949]),
 'fit_time': (3.9625444412231445,
  4.007352113723755,
  4.16746187210083,
  4.135302305221558,
  5.212366104125977),
 'test_time': (0.16793560981750488,
  0.17036819458007812,
  0.12801265716552734,
  0.17074275016784668,
  0.3708791732788086)}

## Definición de funciones
Usaremos estas funciones para evaluar varios algoritmos de recomendación. La función `get_results` escribirá sobre el fichero `results_user_cf.csv` añadiéndole texto al final. Queremos asegurarnos de que el fichero exista y esté vacío (salvo por una cabecera) antes de llamar a dicha función por primera vez.

In [3]:
# Creamos el fichero en el mismo directorio que está guardado este notebook y le añadimos una cabecera
f = open("results_user_cf.csv", 'w')
f.write("K,Algorithm,Precision,Recall,F1\n")
f.close()

In [4]:
# Evaluation extracted from surprise: 
# https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
def measures_at_k(predictions, k, th_recom, th_relev):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    onehits = dict()
    mrr = dict()
    
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= th_relev) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= th_recom) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= th_relev) and (est >= th_recom))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
       
        
    return precisions, recalls


In [5]:
def f1(precision, recall):
    """
        Funcion que calcula el f1 (media armónica) en funcion de precision y recall
    """
    denominador = precision + recall
    
    if denominador == 0:
        return 0
    else:
        return (2 * precision * recall) / denominador

In [6]:
# Guarda los resultados de las recomendaciones en el fichero "results_user_cf.csv"
def get_results(recommendations, k, knn):
    """
        Function to get the measures results 
    """
    # threshold = 4 --> solo se tienen en cuenta peliculas que hayan 
    # sido puntuadas con 4 o 5 estrellas
    precisions, recalls  = measures_at_k(recommendations, k, th_recom=4, th_relev=1)
    
    # Measures can then be averaged over all users
    precision_result = sum(prec for prec in precisions.values()) / len(precisions)
    recall_result = sum(rec for rec in recalls.values()) / len(recalls)
    # Media armónica  
    f1_result = f1(precision_result, recall_result)
    # En este archivo se volcarán los resultados de las métricas. Tiene que existir. 
    f = open("results_user_cf.csv", 'a')
    #f = open("C:/hlocal/results_user_cf.csv", 'a')
    f.write(str(k) + ',' + knn + "," + str(precision_result) + ',' + str(recall_result) + ',' +  str(f1_result) +  '\n') 
    f.close()
    

## Evaluación de los algoritmos de recomendación
Hemos cargado antes los datos de movieLens para 100K con la siguiente función:
```python
data = Dataset.load_builtin('ml-100k')
```
Ahora creamos 2 conjuntos de datos: los datos de entrenamiento (`training_set`) y los de evaluación (`evaluation_set`). Cada uno contendrá la mitad de los datos.

In [7]:
training_set, evaluation_set = train_test_split(data, test_size=.5)

Ahora vamos a emplear varios algoritmos de recomendación y guardar sus resultados en el fichero `results_user_cf.csv`.
### KNN Basic

In [8]:
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [9]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [10]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [11]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

### KNN With Means

In [12]:
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [13]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [14]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [15]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

### KNN With Z-Score

In [16]:
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [17]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [18]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [19]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

### KNN Baseline

In [20]:
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

Aplico el algoritmo sobre el `training_set`.

In [21]:
recommendation_algorithm.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Aplico el algoritmo sobre el `evaluation_set` y obtengo las predicciones en las recomendaciones.

In [22]:
recommendations = recommendation_algorithm.test(evaluation_set)

In [23]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

## Ejercicio:  se pide ejecutar, comprender y escribir comentarios razonados sobre la evaluación de distintos recomendadores.
    
Prueba otros algoritmos de predicción de ratings basados en la estimación de los vecinos más próximos y realiza evaluaciones de su comportamiento. Comenta los resultados.
Puedes consultar la documentación en https://surprise.readthedocs.io/en/stable/knn_inspired.html#

Si vemos el fichero `results_user_cf.csv`, observamos que la precisión de cada algoritmo no varía en función del valor de K ( **¿es esto un fallo?** ), pero el recall sí que aumenta al incrementarse K. Consecuentemente, la media armónica también aumenta y conseguimos mejores recomendaciones.

A la hora de comparar los algoritmos entre sí el que consigue mejor precisión es KNNBasic seguido de, en orden decreciente de precisión, KNN Baseline, KNN With Z-Score y KNN With Means. Es decir, KNN Basic es el algoritmo que mayor proporción de items relevantes consigue de los items que recomienda. No obstante, KNN Basic consigue el peor recall de todos los algoritmos que hemos probado, por lo que es el que menos items recomienda de los items relevantes. El resto tienen recalls parecidos para el mismo valor de K.

Para comparar todos los algoritmos es preferible usar la media armónica de la precisión y el recall pues cuando se incrementa la precisión decrece el recall y viceversa, así que buscamos un "equilibrio" óptimo. En este aspecto podemos observar que KNN Baseline es el algoritmo que mejor se comporta de todos, seguido de KNN With Means y KNN With Z-Score, y siendo KNN Basic el peor de todos.